### **Particle Size Distribution (PSD)**

**Definition**: Particle size distribution refers to the range and frequency of particle sizes in a given sample. It is typically represented as a list of values or a mathematical function that defines the relative amount of particles present according to size.

### **Importance of Particle Size Distribution**
- **Quality and Performance**: Particle size influences many properties of particulate materials, such as flow, compaction, dissolution rates, and stability of suspensions and emulsions.
- **Applications**: PSD is crucial in various industries, including pharmaceuticals, food, cosmetics, and materials science, where controlling particle size can affect product quality and efficacy.

### **Measurement Techniques**
1. **Laser Diffraction**: Uses laser light scattering to measure particle size.
2. **Dynamic Light Scattering**: Measures fluctuations in light scattering due to particle movement.
3. **Image Analysis**: Captures images of particles and analyzes their size and shape.
4. **Centrifugal Sedimentation**: Measures particle size based on their sedimentation rate in a liquid.

### **Interpreting PSD Data**
- **Central Values**: Mean, median, and mode are used


# 🧪 Interactive Particle Size Distribution (PSD) Tool

This notebook provides an interactive tool for analyzing **Particle Size Distribution (PSD)** in soil mechanics. It is designed for educational use and includes:

- 📥 Input fields for sieve sizes and percent passing
- 📊 Real-time PSD curve plotting
- 📐 Calculation of D10, D30, D60
- 📈 Coefficients of Uniformity (Cu) and Curvature (Cc)
- 🧠 Soil gradation hints based on Cu and Cc

---


In [10]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, Markdown

def compute_d_values(sieve_sizes, percent_passing):
    D10 = np.interp(10, percent_passing[::-1], sieve_sizes[::-1])
    D30 = np.interp(30, percent_passing[::-1], sieve_sizes[::-1])
    D60 = np.interp(60, percent_passing[::-1], sieve_sizes[::-1])
    return D10, D30, D60

def plot_psd(sieve_sizes, percent_passing, D10, D30, D60):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sieve_sizes, y=percent_passing, mode='lines+markers', name='PSD Curve'))
    fig.add_trace(go.Scatter(x=[D10], y=[10], mode='markers+text', name='D10',
                             text=["D10"], textposition="top center", marker=dict(size=10, color='red')))
    fig.add_trace(go.Scatter(x=[D30], y=[30], mode='markers+text', name='D30',
                             text=["D30"], textposition="top center", marker=dict(size=10, color='green')))
    fig.add_trace(go.Scatter(x=[D60], y=[60], mode='markers+text', name='D60',
                             text=["D60"], textposition="top center", marker=dict(size=10, color='blue')))
    fig.update_layout(title='Particle Size Distribution Curve',
                      xaxis_title='Sieve Size (mm)',
                      yaxis_title='Percent Passing (%)',
                      xaxis_type='log',
                      xaxis=dict(autorange='reversed'),
                      yaxis=dict(autorange='reversed'),
                      width=800, height=500)
    fig.show()

def classify_soil_by_gradation(Cu, Cc):
    if Cu >= 4 and 1 <= Cc <= 3:
        return "Well-graded gravel (GW)"
    elif Cu >= 6 and 1 <= Cc <= 3:
        return "Well-graded sand (SW)"
    else:
        return "Poorly graded soil (GP/SP)"

sieve_input = widgets.Text(
    value='4.75, 2.0, 1.0, 0.425, 0.25, 0.075',
    description='Sieve Sizes (mm):',
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'}
)

passing_input = widgets.Text(
    value='100, 90, 70, 50, 30, 10',
    description='Percent Passing (%):',
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'}
)

output = widgets.Output()

def update_plot(change=None):
    output.clear_output()
    with output:
        try:
            sieve_sizes = np.array([float(x.strip()) for x in sieve_input.value.split(',')])
            percent_passing = np.array([float(x.strip()) for x in passing_input.value.split(',')])
            if len(sieve_sizes) != len(percent_passing):
                display(Markdown("❌ **Error:** Number of sieve sizes and percent passing values must match."))
                return
            sorted_indices = np.argsort(sieve_sizes)[::-1]
            sieve_sizes = sieve_sizes[sorted_indices]
            percent_passing = percent_passing[sorted_indices]
            D10, D30, D60 = compute_d_values(sieve_sizes, percent_passing)
            Cu = D60 / D10 if D10 != 0 else np.nan
            Cc = (D30**2) / (D10 * D60) if D10 != 0 and D60 != 0 else np.nan
            classification = classify_soil_by_gradation(Cu, Cc)
            display(Markdown(f"### 📐 Calculated Parameters"))
            display(Markdown(f"- **D10:** {D10:.3f} mm  - **D30:** {D30:.3f} mm  - **D60:** {D60:.3f} mm"))
            display(Markdown(f"- **Coefficient of Uniformity (Cu):** {Cu:.2f}  - **Coefficient of Curvature (Cc):** {Cc:.2f}"))
            display(Markdown(f"### 🧠 Soil Gradation Hint: **{classification}**"))
            plot_psd(sieve_sizes, percent_passing, D10, D30, D60)
        except Exception as e:
            display(Markdown(f"❌ **Error:** {e}"))

sieve_input.observe(update_plot, names='value')
passing_input.observe(update_plot, names='value')

display(widgets.VBox([sieve_input, passing_input]), output)
update_plot()


import numpy as np

def classify_soil_from_psd(sieve_sizes, percent_passing):
    sorted_indices = np.argsort(sieve_sizes)[::-1]
    sieve_sizes = np.array(sieve_sizes)[sorted_indices]
    percent_passing = np.array(percent_passing)[sorted_indices]

    D10 = np.interp(10, percent_passing[::-1], sieve_sizes[::-1])
    D30 = np.interp(30, percent_passing[::-1], sieve_sizes[::-1])
    D60 = np.interp(60, percent_passing[::-1], sieve_sizes[::-1])

    Cu = D60 / D10 if D10 != 0 else np.nan
    Cc = (D30 ** 2) / (D10 * D60) if D10 != 0 and D60 != 0 else np.nan

    fines_index = np.where(np.array(sieve_sizes) <= 0.075)[0]
    percent_fines = percent_passing[fines_index[0]] if fines_index.size > 0 else 0

    if percent_fines > 50:
        classification = "Fine-grained soil (Use Atterberg limits for further classification)"
    elif percent_fines > 12:
        classification = "Coarse-grained soil with fines (e.g., SC, SM, GC, GM)"
    else:
        if D10 == 0 or D60 == 0:
            classification = "Insufficient data for classification"
        elif Cu >= 4 and 1 <= Cc <= 3:
            classification = "Well-graded gravel (GW)" if sieve_sizes[0] > 4.75 else "Well-graded sand (SW)"
        else:
            classification = "Poorly graded gravel (GP)" if sieve_sizes[0] > 4.75 else "Poorly graded sand (SP)"

    return {
        'D10 (mm)': round(D10, 3),
        'D30 (mm)': round(D30, 3),
        'D60 (mm)': round(D60, 3),
        'Cu': round(Cu, 2),
        'Cc': round(Cc, 2),
        'Percent Fines': round(percent_fines, 1),
        'Classification': classification
    }

# Example dataset
sieve_sizes = [4.75, 2.0, 1.0, 0.425, 0.25, 0.075]
percent_passing = [100, 90, 70, 50, 30, 10]

# Run classification
result = classify_soil_from_psd(sieve_sizes, percent_passing)

# Display results
for key, value in result.items():
    print(f"{key}: {value}")


Output()

D10 (mm): 0.075
D30 (mm): 0.25
D60 (mm): 0.712
Cu: 9.5
Cc: 1.17
Percent Fines: 10
Classification: Well-graded sand (SW)


## Reflective Questions

1. **How does particle size distribution affect the properties of materials in different industries?**
2. **What are the advantages and limitations of different particle size measurement techniques?**
3. **How can understanding particle size distribution improve product quality and performance?**
4. **What challenges might arise when interpreting particle size distribution data?**

## Conceptual Questions

1. **Define particle size distribution and explain its significance in material science.**
2. **Describe the process of laser diffraction and how it is used to measure particle size.**
3. **Explain the importance of D10, D30, and D60 values in particle size distribution analysis.**
4. **What is the difference between well-graded and poorly graded soils? How are these classifications determined?**

## Quiz




In [13]:

import numpy as np
import ipywidgets as widgets
from IPython.display import display, Markdown

# Quiz Questions
questions = [
    {
        "question": "What does D10 represent in particle size distribution?",
        "options": ["The diameter at which 10% of the sample's mass is finer", "The diameter at which 10% of the sample's mass is coarser", "The average particle size", "The largest particle size"],
        "answer": "The diameter at which 10% of the sample's mass is finer"
    },
    {
        "question": "Which technique uses laser light scattering to measure particle size?",
        "options": ["Dynamic Light Scattering", "Laser Diffraction", "Image Analysis", "Centrifugal Sedimentation"],
        "answer": "Laser Diffraction"
    },
    {
        "question": "What is the Coefficient of Uniformity (Cu) used for?",
        "options": ["To determine the average particle size", "To classify the soil gradation", "To measure the particle shape", "To calculate the particle density"],
        "answer": "To classify the soil gradation"
    },
    {
        "question": "What does a high percentage of fines indicate about the soil?",
        "options": ["The soil is well-graded", "The soil is poorly graded", "The soil is fine-grained", "The soil is coarse-grained"],
        "answer": "The soil is fine-grained"
    }
]

# Function to create quiz
def create_quiz(questions):
    quiz_widgets = []
    for q in questions:
        question_widget = widgets.VBox([
            widgets.HTML(value=f"<b>{q['question']}</b>"),
            widgets.RadioButtons(options=q['options'], layout={'width': 'max-content'})
        ])
        quiz_widgets.append(question_widget)
    return quiz_widgets

# Function to check answers
def check_answers(quiz_widgets, questions):
    score = 0
    for i, widget in enumerate(quiz_widgets):
        selected = widget.children[1].value
        if selected == questions[i]['answer']:
            score += 1
    return score

# Display quiz
quiz_widgets = create_quiz(questions)
display(widgets.VBox(quiz_widgets))

# Button to submit answers
submit_button = widgets.Button(description="Submit Answers")
output = widgets.Output()

def on_submit(b):
    with output:
        output.clear_output()
        score = check_answers(quiz_widgets, questions)
        display(Markdown(f"### Your Score: {score}/{len(questions)}"))

submit_button.on_click(on_submit)
display(submit_button, output)

Button(description='Submit Answers', style=ButtonStyle())

Output()